# エージェントのベンチマーキング: 検索 + 計算機

ここでは、計算機と検索ツールにアクセスできるタスクでエージェントのパフォーマンスをベンチマークする方法について説明します。

トレースを有効にして評価/ベンチマークを行うことを強くお勧めします。トレースとは何か、どのように設定するかについては[こちら](https://langchain.readthedocs.io/en/latest/tracing.html)をご覧ください。

In [ ]:
# トレーシングを使用しない場合は、コメントアウトしてください。
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

## データの読み込み
まずは、データを読み込みましょう。

In [ ]:
from langchain.evaluation.loading import load_dataset
dataset = load_dataset("agent-search-calculator")

## チェーンの設定
次に、これらの質問に答えることができるエージェントをロードする必要があります。

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMMathChain
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.agents import AgentType

tools = load_tools(['serpapi', 'llm-math'], llm=OpenAI(temperature=0))
agent = initialize_agent(tools, OpenAI(temperature=0), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

## 予測を行う

まず、私たちは1つのデータポイントごとに予測を行うことができます。この粒度で行うことで、出力を詳細に探索することができます。また、複数のデータポイントに対して実行するよりもコストを抑えることができます。

In [ ]:
print(dataset[0]['question'])
agent.run(dataset[0]['question'])

## たくさんの予測を作る
これで予測を作成することができます。

In [ ]:
agent.run(dataset[4]['question'])

In [ ]:
predictions = []
predicted_dataset = []
error_dataset = []
for data in dataset:
    new_data = {"input": data["question"], "answer": data["answer"]}
    try:
        predictions.append(agent(new_data))
        predicted_dataset.append(new_data)
    except Exception as e:
        predictions.append({"output": str(e), **new_data})
        error_dataset.append(new_data)

## 性能評価
予測結果を評価できるようになりました。最初に行うことは、目視で予測結果を確認することです。

In [ ]:
predictions[0]

次に、言語モデルを使用して、それらをプログラム的にスコアリングすることができます。

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(dataset, predictions, question_key="question", prediction_key="output")

`predictions` 辞書に評価済みの出力を追加し、そのグレードの数をカウントすることができます。

In [ ]:
for i, prediction in enumerate(predictions):
    prediction['grade'] = graded_outputs[i]['text']

In [ ]:
from collections import Counter
Counter([pred['grade'] for pred in predictions])

間違った例をフィルタリングして、それらを見ることもできます。

In [ ]:
incorrect = [pred for pred in predictions if pred['grade'] == " INCORRECT"]

In [ ]:
incorrect